# Approches collaboratives : utilisateur-utilisateur, item-item

Pierrick DOSSIN  
Guillaume RIU

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import random
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

## Chargement des Données

In [2]:
votes = pd.read_csv('Data/votes.csv')

# Matrice Utilisateur Item
MUI = votes.pivot(index="user.id", columns="item.id", values="rating")
MUI_numpy = MUI.to_numpy()
MUI_numpy_flat = MUI_numpy.reshape(-1)

MUI.head()

item.id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user.id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Définition des métriques

In [3]:
# Erreur quadratique moyenne
def MSE_mat(y_pred, y_true):
    return np.nanmean((y_pred - y_true)**2)

# Erreur absolue moyenne
def MAE_mat(y_pred, y_true):
    return np.nanmean(np.abs(y_pred - y_true))

In [4]:
def recommandations_list(votes_predits, nmb_reco):
    # liste des meilleurs recommandations n'ayant pas déjà été vues
    return np.argsort(np.array(-votes_predits * np.isnan(MUI)))[:,:nmb_reco]

In [5]:
def diversite(votes_observes, votes_predits, nmb_reco):
    diversite = 0
    reco = recommandations_list(votes_predits, nmb_reco)
    votes_observes = votes_observes.replace(np.nan, 0)
    # Matrice des similarités item selon les votes observés
    cosine_similarity = 1 - pairwise_distances(votes_observes.T, metric="cosine")
    for u in range(votes_observes.shape[0]):
        for i in range(nmb_reco):
            for j in range(nmb_reco):
                if i!= j:
                    diversite += cosine_similarity[reco[u][i], reco[u][j]]
    diversite /= nmb_reco * (nmb_reco - 1) * votes_observes.shape[0]
    return diversite

In [6]:
def nouveaute(votes_observes, votes_predits, nmb_reco):
    nouveaute = 0
    reco = recommandations_list(votes_predits, nmb_reco)
    MUI_zero_one = votes_observes.replace(np.nan, 0)
    MUI_zero_one[MUI_zero_one > 0] = 1
    proportion_visionnage_item = np.nanmean(MUI_zero_one, axis=0)
    for u in range(votes_observes.shape[0]):
        for i in range(nmb_reco):
            nouveaute += -np.log2(proportion_visionnage_item[reco[u][i]])
    nouveaute /= nmb_reco * len(votes_observes)
    return nouveaute

In [7]:
def couverture(votes_observes, votes_predits, nmb_reco):
    reco = recommandations_list(votes_predits, nmb_reco)
    nb_item_reco = len(np.unique(reco))
    nb_item = votes_observes.shape[1]
    couverture = nb_item_reco / nb_item
    return couverture

In [8]:
def utilite(votes_observes, votes_predits, nmb_reco):
    # Moyenne des meilleurs recommandations
    reco = np.sort(np.array(-votes_predits * np.isnan(MUI)))[:,:nmb_reco]
    return -np.mean(reco)

In [19]:
def p(MUI_nan, i, j = None):
    if (j==None):
        nan = sum(MUI_nan[:,i]) 
        return (MUI.shape[0] - nan) / MUI.shape[0]
    else:
        nan = sum(MUI_nan[:,i] * MUI_nan[:,j])
        return (MUI.shape[0] - nan) / MUI.shape[0]

In [22]:
def aime_list(votes_observes, nmb_reco):
    MUI_zero_one = votes_observes.replace(np.nan, 0)
    return np.argsort(np.array(-MUI_zero_one))[:,:nmb_reco]

In [20]:
def serendipite(votes_observes, votes_predits, nmb_reco):
    MUI_nan = np.array(np.isnan(votes_observes))
    reco = recommandations_list(votes_predits, nmb_reco)
    aime = aime_list(votes_observes, nmb_reco)
    pmi = 0
    for u in range(votes_observes.shape[0]):
        for i in reco[u]:
            for j in aime[u]:
                pmi += np.log2(p(MUI_nan, i,j)/(p(MUI_nan, i)*p(MUI_nan, j)))
    pmi /= nmb_reco * nmb_reco * votes_observes.shape[0]
    return pmi

In [21]:
def calcul_facteur(votes_observes, votes_predits, facteur, nmb_reco):
    if facteur == "serendipite":
        return serendipite(votes_observes, votes_predits, nmb_reco)
    elif facteur == "utilite":
        return utilite(votes_observes, votes_predits, nmb_reco)
    elif facteur == "couverture":
        return couverture(votes_observes, votes_predits, nmb_reco)
    elif facteur == "nouveaute":
        return nouveaute(votes_observes, votes_predits, nmb_reco)
    elif facteur == "diversite":
        return diversite(votes_observes, votes_predits, nmb_reco)


## Prédictions

In [13]:
# Création des indices pour les valeurs différentes de np.nan
indices = np.arange(0, MUI_numpy.shape[0]*MUI_numpy.shape[1])
indices_na = indices[~np.isnan(MUI_numpy_flat)]

# Split Train Test des indices
nb_replis = 5
np.random.shuffle(indices_na)
idx_split = np.split(indices_na, nb_replis)

In [26]:
# Liste des erreurs MSE et MAE pour chacun des 5 plis de la validation croisée

MSE_votes_sans_biais_u = []
MSE_votes_avec_biais_u = []
MSE_votes_voisins_rapproches_u = []

MSE_votes_sans_biais_i = []
MSE_votes_avec_biais_i = []
MSE_votes_voisins_rapproches_i = []

MAE_votes_sans_biais_u = []
MAE_votes_avec_biais_u = []
MAE_votes_voisins_rapproches_u = []

MAE_votes_sans_biais_i = []
MAE_votes_avec_biais_i = []
MAE_votes_voisins_rapproches_i = []

# Liste des facteurs de succès pour chacun des 5 plis de la validation croisée

serendipite_votes_sans_biais_u = []
serendipite_votes_avec_biais_u = []
serendipite_votes_voisins_rapproches_u = []

serendipite_votes_sans_biais_i = []
serendipite_votes_avec_biais_i = []
serendipite_votes_voisins_rapproches_i = []

utilite_votes_sans_biais_u = []
utilite_votes_avec_biais_u = []
utilite_votes_voisins_rapproches_u = []

utilite_votes_sans_biais_i = []
utilite_votes_avec_biais_i = []
utilite_votes_voisins_rapproches_i = []

couverture_votes_sans_biais_u = []
couverture_votes_avec_biais_u = []
couverture_votes_voisins_rapproches_u = []

couverture_votes_sans_biais_i = []
couverture_votes_avec_biais_i = []
couverture_votes_voisins_rapproches_i = []

nouveaute_votes_sans_biais_u = []
nouveaute_votes_avec_biais_u = []
nouveaute_votes_voisins_rapproches_u = []

nouveaute_votes_sans_biais_i = []
nouveaute_votes_avec_biais_i = []
nouveaute_votes_voisins_rapproches_i = []

diversite_votes_sans_biais_u = []
diversite_votes_avec_biais_u = []
diversite_votes_voisins_rapproches_u = []

diversite_votes_sans_biais_i = []
diversite_votes_avec_biais_i = []
diversite_votes_voisins_rapproches_i = []


for i in range(nb_replis):
    
    # Liste d'indice train et test
    idx_train = np.delete(idx_split, i, axis=0).flatten()
    idx_test  = idx_split[i]

    # On enlève les valeurs de test de la matrice d'entrainement
    MUI_numpy_flat_train = MUI_numpy_flat.copy()
    MUI_numpy_flat_test  = MUI_numpy_flat.copy()
    MUI_numpy_flat_train[idx_test] = np.nan
    MUI_numpy_flat_test[idx_train] = np.nan

    MUI_train = pd.DataFrame(MUI_numpy_flat_train.reshape(MUI_numpy.shape))
    MUI_test  = pd.DataFrame(MUI_numpy_flat_test.reshape(MUI_numpy.shape))

    MUI_train_zero_one = MUI_train.replace(np.nan, 0)
    MUI_train_zero_one[MUI_train_zero_one > 0] = 1

    MUI_train_zero = MUI_train.replace(np.nan, 0)


    # Vote sans correction biais

    w_u = 1 - pairwise_distances(MUI_train_zero, metric="cosine")
    w_i = 1 - pairwise_distances(MUI_train_zero.T, metric="cosine")

    pred_sans_biais_u = w_u.dot(MUI_train_zero) / abs(w_u).dot(MUI_train_zero_one)
    pred_sans_biais_i = np.array(MUI_train_zero.dot(w_i) / MUI_train_zero_one.dot(abs(w_i)))

    # Vote avec correction biais

    MUI_train_means_U = np.expand_dims(np.nanmean(MUI_train, axis=1), axis=-1)
    MUI_train_norm_U = (MUI_train - MUI_train_means_U).replace(np.nan, 0)

    MUI_train_means_I = np.expand_dims(np.nanmean(MUI_train, axis=0), axis=0)
    MUI_train_norm_I = (MUI_train - MUI_train_means_I).replace(np.nan, 0)

    pred_avec_biais_u = w_u.dot(MUI_train_norm_U) / abs(w_u).dot(MUI_train_zero_one) + MUI_train_means_U
    pred_avec_biais_i = np.array(MUI_train_norm_I.dot(w_i) / MUI_train_zero_one.dot(abs(w_i)) + MUI_train_means_I)

    # Vote 100 voisins rapprochés

    w_u_100_neighboors = w_u.copy()
    w_i_100_neighboors = w_i.copy()

    for u in w_u_100_neighboors:
        ind = np.argpartition(u, -100)[:-100]
        u[ind] = 0

    for i in w_i_100_neighboors:
        ind = np.argpartition(i, -100)[:-100]
        i[ind] = 0

    w_i_100_neighboors = w_i_100_neighboors.T

    pred_100_neighboors_u = w_u_100_neighboors.dot(MUI_train_norm_U) / abs(w_u_100_neighboors).dot(MUI_train_zero_one) + MUI_train_means_U
    pred_100_neighboors_i = np.array(MUI_train_norm_I.dot(w_i_100_neighboors) / MUI_train_zero_one.dot(abs(w_i_100_neighboors)) + MUI_train_means_I)


    MSE_votes_sans_biais_u.append(MSE_mat(pred_sans_biais_u, MUI_test))
    MAE_votes_sans_biais_u.append(MAE_mat(pred_sans_biais_u, MUI_test))

    MSE_votes_avec_biais_u.append(MSE_mat(pred_avec_biais_u, MUI_test))
    MAE_votes_avec_biais_u.append(MAE_mat(pred_avec_biais_u, MUI_test))

    MSE_votes_voisins_rapproches_u.append(MSE_mat(pred_100_neighboors_u, MUI_test))
    MAE_votes_voisins_rapproches_u.append(MAE_mat(pred_100_neighboors_u, MUI_test))

    MSE_votes_sans_biais_i.append(MSE_mat(pred_sans_biais_i, MUI_test))
    MAE_votes_sans_biais_i.append(MAE_mat(pred_sans_biais_i, MUI_test))

    MSE_votes_avec_biais_i.append(MSE_mat(pred_avec_biais_i, MUI_test))
    MAE_votes_avec_biais_i.append(MAE_mat(pred_avec_biais_i, MUI_test))

    MSE_votes_voisins_rapproches_i.append(MSE_mat(pred_100_neighboors_i, MUI_test))
    MAE_votes_voisins_rapproches_i.append(MAE_mat(pred_100_neighboors_i, MUI_test))


    serendipite_votes_sans_biais_u.append(calcul_facteur(MUI, pred_sans_biais_u, "serendipite", 10))
    utilite_votes_sans_biais_u.append(calcul_facteur(MUI, pred_sans_biais_u, "utilite", 10))
    couverture_votes_sans_biais_u.append(calcul_facteur(MUI, pred_sans_biais_u, "couverture", 10))
    nouveaute_votes_sans_biais_u.append(calcul_facteur(MUI, pred_sans_biais_u, "nouveaute", 10))
    diversite_votes_sans_biais_u.append(calcul_facteur(MUI, pred_sans_biais_u, "diversite", 10))

    serendipite_votes_avec_biais_u.append(calcul_facteur(MUI, pred_avec_biais_u, "serendipite", 10))
    utilite_votes_avec_biais_u.append(calcul_facteur(MUI, pred_avec_biais_u, "utilite", 10))
    couverture_votes_avec_biais_u.append(calcul_facteur(MUI, pred_avec_biais_u, "couverture", 10))
    nouveaute_votes_avec_biais_u.append(calcul_facteur(MUI, pred_avec_biais_u, "nouveaute", 10))
    diversite_votes_avec_biais_u.append(calcul_facteur(MUI, pred_avec_biais_u, "diversite", 10))

    serendipite_votes_voisins_rapproches_u.append(calcul_facteur(MUI, pred_100_neighboors_u, "serendipite", 10))
    utilite_votes_voisins_rapproches_u.append(calcul_facteur(MUI, pred_100_neighboors_u, "utilite", 10))
    couverture_votes_voisins_rapproches_u.append(calcul_facteur(MUI, pred_100_neighboors_u, "couverture", 10))
    nouveaute_votes_voisins_rapproches_u.append(calcul_facteur(MUI, pred_100_neighboors_u, "nouveaute", 10))
    diversite_votes_voisins_rapproches_u.append(calcul_facteur(MUI, pred_100_neighboors_u, "diversite", 10))

    serendipite_votes_sans_biais_i.append(calcul_facteur(MUI, pred_sans_biais_i, "serendipite", 10))
    utilite_votes_sans_biais_i.append(calcul_facteur(MUI, pred_sans_biais_i, "utilite", 10))
    couverture_votes_sans_biais_i.append(calcul_facteur(MUI, pred_sans_biais_i, "couverture", 10))
    nouveaute_votes_sans_biais_i.append(calcul_facteur(MUI, pred_sans_biais_i, "nouveaute", 10))
    diversite_votes_sans_biais_i.append(calcul_facteur(MUI, pred_sans_biais_i, "diversite", 10))

    serendipite_votes_avec_biais_i.append(calcul_facteur(MUI, pred_avec_biais_i, "serendipite", 10))
    utilite_votes_avec_biais_i.append(calcul_facteur(MUI, pred_avec_biais_i, "utilite", 10))
    couverture_votes_avec_biais_i.append(calcul_facteur(MUI, pred_avec_biais_i, "couverture", 10))
    nouveaute_votes_avec_biais_i.append(calcul_facteur(MUI, pred_avec_biais_i, "nouveaute", 10))
    diversite_votes_avec_biais_i.append(calcul_facteur(MUI, pred_avec_biais_i, "diversite", 10))

    serendipite_votes_voisins_rapproches_i.append(calcul_facteur(MUI, pred_100_neighboors_i, "serendipite", 10))
    utilite_votes_voisins_rapproches_i.append(calcul_facteur(MUI, pred_100_neighboors_i, "utilite", 10))
    couverture_votes_voisins_rapproches_i.append(calcul_facteur(MUI, pred_100_neighboors_i, "couverture", 10))
    nouveaute_votes_voisins_rapproches_i.append(calcul_facteur(MUI, pred_100_neighboors_i, "nouveaute", 10))
    diversite_votes_voisins_rapproches_i.append(calcul_facteur(MUI, pred_100_neighboors_i, "diversite", 10))


C:\Users\pierr\AppData\Local\Temp\ipykernel_2236\2320157975.py:88: RuntimeWarning: invalid value encountered in divide
  pred_sans_biais_u = w_u.dot(MUI_train_zero) / abs(w_u).dot(MUI_train_zero_one)
C:\Users\pierr\AppData\Local\Temp\ipykernel_2236\2320157975.py:96: RuntimeWarning: Mean of empty slice
  MUI_train_means_I = np.expand_dims(np.nanmean(MUI_train, axis=0), axis=0)
C:\Users\pierr\AppData\Local\Temp\ipykernel_2236\2320157975.py:99: RuntimeWarning: invalid value encountered in divide
  pred_avec_biais_u = w_u.dot(MUI_train_norm_U) / abs(w_u).dot(MUI_train_zero_one) + MUI_train_means_U
C:\Users\pierr\AppData\Local\Temp\ipykernel_2236\2320157975.py:117: RuntimeWarning: invalid value encountered in divide
  pred_100_neighboors_u = w_u_100_neighboors.dot(MUI_train_norm_U) / abs(w_u_100_neighboors).dot(MUI_train_zero_one) + MUI_train_means_U
C:\Users\pierr\AppData\Local\Temp\ipykernel_2236\2320157975.py:88: RuntimeWarning: invalid value encountered in divide
  pred_sans_biais_u = w

In [32]:
print("Erreur approche utilisateur-utilisateur sans correction de biais :")
print("MSE: ", np.mean(MSE_votes_sans_biais_u))
print("MAE: ", np.mean(MAE_votes_sans_biais_u), "\n")

print("Erreur approche item-item sans correction de biais :")
print("MSE: ", np.mean(MSE_votes_sans_biais_i))
print("MAE: ", np.mean(MAE_votes_sans_biais_i), "\n")

print("Erreur approche utilisateur-utilisateur avec correction de biais :")
print("MSE: ", np.mean(MSE_votes_avec_biais_u))
print("MAE: ", np.mean(MAE_votes_avec_biais_u), "\n")

print("Erreur approche item-item avec correction de biais :")
print("MSE: ", np.mean(MSE_votes_avec_biais_i))
print("MAE: ", np.mean(MAE_votes_avec_biais_i), "\n")

print("Erreur approche utilisateur-utilisateur avec 100 voisins ajoutés :")
print("MSE: ", np.mean(MSE_votes_voisins_rapproches_u))
print("MAE: ", np.mean(MAE_votes_voisins_rapproches_u), "\n")

print("Erreur approche item-item avec 100 voisins ajoutés :")
print("MSE: ", np.mean(MSE_votes_voisins_rapproches_i))
print("MAE: ", np.mean(MAE_votes_voisins_rapproches_i), "\n")

Erreur approche utilisateur-utilisateur sans correction de biais :
MSE:  1.0325181206939114
MAE:  0.8102502828117579 

Erreur approche item-item sans correction de biais :
MSE:  1.025958473178745
MAE:  0.8061226374608481 

Erreur approche utilisateur-utilisateur avec correction de biais :
MSE:  0.9066836402736946
MAE:  0.7498701160080046 

Erreur approche item-item avec correction de biais :
MSE:  0.8699419364048577
MAE:  0.7354475211629607 

Erreur approche utilisateur-utilisateur avec 100 voisins ajoutés :
MSE:  0.8999437846171657
MAE:  0.7424838568840044 

Erreur approche item-item avec 100 voisins ajoutés :
MSE:  0.8430124287359273
MAE:  0.7212305387978677 



In [31]:
print("Facteur de succès de approche utilisateur-utilisateur sans correction de biais :")
print("utilite = ", np.mean(utilite_votes_sans_biais_u))
print("diversite = ", np.mean(diversite_votes_sans_biais_u))
print("nouveaute = ", np.mean(nouveaute_votes_sans_biais_u))
print("serendipite = ", np.mean(serendipite_votes_sans_biais_u))
print("couverture = ", np.mean(couverture_votes_sans_biais_u), "\n")

print("Facteur de succès de approche item-item sans correction de biais :")
print("utilite = ", np.mean(utilite_votes_sans_biais_i))
print("diversite = ", np.mean(diversite_votes_sans_biais_i))
print("nouveaute = ", np.mean(nouveaute_votes_sans_biais_i))
print("serendipite = ", np.mean(serendipite_votes_sans_biais_i))
print("couverture = ", np.mean(couverture_votes_sans_biais_i), "\n")

print("Facteur de succès de approche utilisateur-utilisateur avec correction de biais :")
print("utilite = ", np.mean(utilite_votes_avec_biais_u))
print("diversite = ", np.mean(diversite_votes_avec_biais_u))
print("nouveaute = ", np.mean(nouveaute_votes_avec_biais_u))
print("serendipite = ", np.mean(serendipite_votes_avec_biais_u))
print("couverture = ", np.mean(couverture_votes_avec_biais_u), "\n")

print("Facteur de succès de approche item-item avec correction de biais :")
print("utilite = ", np.mean(utilite_votes_avec_biais_i))
print("diversite = ", np.mean(diversite_votes_avec_biais_i))
print("nouveaute = ", np.mean(nouveaute_votes_avec_biais_i))
print("serendipite = ", np.mean(serendipite_votes_avec_biais_i))
print("couverture = ", np.mean(couverture_votes_avec_biais_i), "\n")

print("Facteur de succès de approche utilisateur-utilisateur avec 100 voisins rapprochés :")
print("utilite = ", np.mean(utilite_votes_voisins_rapproches_u))
print("diversite = ", np.mean(diversite_votes_voisins_rapproches_u))
print("nouveaute = ", np.mean(nouveaute_votes_voisins_rapproches_u))
print("serendipite = ", np.mean(serendipite_votes_voisins_rapproches_u))
print("couverture = ", np.mean(couverture_votes_voisins_rapproches_u), "\n")

print("Facteur de succès de approche item-item avec 100 voisins rapprochés :")
print("utilite = ", np.mean(utilite_votes_voisins_rapproches_i))
print("diversite = ", np.mean(diversite_votes_voisins_rapproches_i))
print("nouveaute = ", np.mean(nouveaute_votes_voisins_rapproches_i))
print("serendipite = ", np.mean(serendipite_votes_voisins_rapproches_i))
print("couverture = ", np.mean(couverture_votes_voisins_rapproches_i))


Facteur de succès de approche utilisateur-utilisateur sans correction de biais :
utilite =  4.994869428287458
diversite =  0.009548450526844958
nouveaute =  9.050521918919912
serendipite =  9.073514535825847
couverture =  0.021521997621878716 

Facteur de succès de approche item-item sans correction de biais :
utilite =  4.363216712205541
diversite =  0.14838623550849156
nouveaute =  9.055951988771664
serendipite =  9.08071872894457
couverture =  0.21712247324613557 

Facteur de succès de approche utilisateur-utilisateur avec correction de biais :
utilite =  5.073155556109294
diversite =  0.027848440463561618
nouveaute =  8.781278528041863
serendipite =  8.809027997411276
couverture =  0.02318668252080856 

Facteur de succès de approche item-item avec correction de biais :
utilite =  5.1330118438166625
diversite =  0.01586651025578459
nouveaute =  8.992629503129919
serendipite =  9.01781906396863
couverture =  0.04684898929845423 

Facteur de succès de approche utilisateur-utilisateur 